Benötigte Module importieren und Datei laden. Die ersten Zeilen werden ausgegeben.

In [ ]:
import pandas as pd
import tensorflow as tf
from keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
from keras.applications import vgg19
import tensorflow_datasets as tfds

Daten vorbereiten.

In [ ]:
base_model = vgg19.VGG19(include_top=False,input_shape=[300,300,3])
base_model.trainable = False
base_model.summary()

Modell um zwei Schichten erweitern

In [ ]:
x = tf.keras.layers.Dropout(0.3)(base_model.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256,activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(3,activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)
model.summary()



Daten herunterladen

In [ ]:
(train_data, train_col), (test_data, test_col) = tfds.as_numpy(tfds.load(
    'rock_paper_scissors',
    split=['train', 'test'],
    batch_size=-1,
    as_supervised=True
))

Ausgabe eines Bildes

In [ ]:
class_names = ['rock', 'paper', 'scissor']
index = 111
plt.imshow(train_data[index])
plt.xlabel(class_names[train_col[index]])
plt.show()

print(train_data.shape)

Training und Test

In [ ]:
cb_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

train_data = tf.keras.applications.vgg19.preprocess_input(train_data)
test_data = tf.keras.applications.vgg19.preprocess_input(test_data)

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy']) 
model.fit(train_data, train_col, epochs=100, validation_data=(test_data, test_col), batch_size=32, shuffle=True, callbacks=[cb_early])

Modell speichern

In [ ]:
model.save('transfer.h5')